# Step 2: Private Prediction using Syft Keras - Serving (Server)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

## Model

In [2]:
task_classes = 10
task_shape = [1, 28, 28, 1]
pre_trained_weights = 'short-conv-mnist.h5'

In [3]:
model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=task_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(task_classes, name="logit"))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


### Load pre-trained weights

In [4]:
model.load_weights(pre_trained_weights)

### Secure the model by sharing the weights

In [5]:
alice = sy.TFEWorker(host='localhost:4000')
bob = sy.TFEWorker(host='localhost:4001')
carol = sy.TFEWorker(host='localhost:4002')

model.share(alice, bob, carol)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



### Serve model

In [6]:
model.serve(num_steps=3)

Served
Served
Served


### Shutdown model when done

In [7]:
model.shutdown()